In [1]:
import pandas as pd
from dfply import *
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D,Dense,Flatten
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split
print(tf.__version__)

2.1.0


In [2]:
df = pd.read_csv("E:\\ProjectData\\ERAU-REU\\Project-Drone-Behavior\\behavior-captures\\behavior-signal-multiple-processed.csv")

In [3]:
#plot = df.plot(legend=False)
#plot.figure.savefig("E:\\ProjectData\\ERAU-REU\\Project-Drone-Behavior\\plots\\allplot.png")

In [4]:
#plot = videoDf.plot(legend=False)
#plot.figure.savefig("E:\\ProjectData\\ERAU-REU\\Project-Drone-Behavior\\plots\\allVideoPlot.png")

In [5]:
df >>= mutate(drone_present = case_when([df.behavior == 'surround','yes'],
[df.behavior == 'straight','yes'],
[df.behavior == 'noise','no']))
presence_labs = df['drone_present']

In [6]:
values = np.array(presence_labs)
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)

onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
y = onehot_encoder.fit_transform(integer_encoded)

y.shape

(3200, 2)

In [7]:
x = df.drop(['behavior','signal','multiple','drone_present'],axis=1).values

In [8]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [9]:
model = keras.Sequential(name='test')
model.add(Dense(64, activation='relu', input_shape=(1280,)))
model.add(Dense(32, activation='relu'))
model.add(Dense(2, activation='sigmoid'))

In [10]:
model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])
model.build(input_shape=(1280,))
model.summary()

Model: "test"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                81984     
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 66        
Total params: 84,130
Trainable params: 84,130
Non-trainable params: 0
_________________________________________________________________


In [11]:
fit1 = model.fit(x = x_train,
        y = y_train, 
        epochs = 100,
        batch_size = 32,
        validation_split = .2,
        verbose=True)

Train on 2048 samples, validate on 512 samples
Epoch 1/100
2048/2048 [==============================] - 1s 351us/sample - loss: 0.6851 - accuracy: 0.4390 - val_loss: 0.6788 - val_accuracy: 0.4512
Epoch 2/100
2048/2048 [==============================] - 0s 62us/sample - loss: 0.6729 - accuracy: 0.5508 - val_loss: 0.6647 - val_accuracy: 0.6309
Epoch 3/100
2048/2048 [==============================] - 0s 57us/sample - loss: 0.6595 - accuracy: 0.6233 - val_loss: 0.6507 - val_accuracy: 0.6406
Epoch 4/100
2048/2048 [==============================] - 0s 55us/sample - loss: 0.6474 - accuracy: 0.6238 - val_loss: 0.6388 - val_accuracy: 0.6426
Epoch 5/100
2048/2048 [==============================] - 0s 53us/sample - loss: 0.6369 - accuracy: 0.6238 - val_loss: 0.6285 - val_accuracy: 0.6426
Epoch 6/100
2048/2048 [==============================] - 0s 56us/sample - loss: 0.6277 - accuracy: 0.6243 - val_loss: 0.6195 - val_accuracy: 0.6426
Epoch 7/100
2048/2048 [==============================] - 0s 51us

In [12]:
prediction = model.predict(x_test)
prediction_df = pd.DataFrame(prediction, columns=['prob_present','prob_not_present']) 

In [13]:
y_test = pd.DataFrame(y_test,columns=['present','not_present'])

y_pred = y_test >> bind_cols(prediction_df)

y_pred >>= mutate(actual = case_when([y_pred.present == 1, 'present'],
[y_pred.not_present == 1, 'not_present']),
predicted = case_when([y_pred.prob_present > y_pred.prob_not_present, 'present'],
[y_pred.prob_present < y_pred.prob_not_present, 'not_present']))

y_pred >>= mutate(correct = case_when([y_pred.actual == y_pred.predicted, 'yes'],
[y_pred.actual != y_pred.predicted, 'no']))
y_pred


,present,not_present,prob_present,prob_not_present,actual,predicted,correct
0,0.0,1.0,0.193071,0.805527,not_present,not_present,yes
1,1.0,0.0,0.221193,0.751699,present,not_present,no
2,0.0,1.0,0.259591,0.707430,not_present,not_present,yes
3,0.0,1.0,0.165224,0.824095,not_present,not_present,yes
4,1.0,0.0,0.191563,0.808546,present,not_present,no
...,...,...,...,...,...,...,...
635,1.0,0.0,0.991199,0.009219,present,present,yes
636,0.0,1.0,0.195044,0.800591,not_present,not_present,yes
637,0.0,1.0,0.173823,0.824452,not_present,not_present,yes
638,1.0,0.0,0.973547,0.021875,present,present,yes


In [20]:
small_df = df[df.columns[0:256]]
small_df >>= bind_cols(df.drone_present)

presence_labs = df_small['drone_present']

values = np.array(presence_labs)
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)

onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
y = onehot_encoder.fit_transform(integer_encoded)

x = df_small.drop(['drone_present'],axis=1).values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

model = keras.Sequential(name='test')
model.add(Dense(64, activation='relu', input_shape=(256,)))
model.add(Dense(32, activation='relu'))
model.add(Dense(2, activation='sigmoid'))

model = keras.Sequential(name='test')
model.add(Dense(64, activation='relu', input_shape=(256,)))
model.add(Dense(32, activation='relu'))
model.add(Dense(2, activation='sigmoid'))

fit1 = model.fit(x = x_train,
        y = y_train, 
        epochs = 100,
        batch_size = 32,
        validation_split = .2,
        verbose=True)

prediction = model.predict(x_test)
prediction_df = pd.DataFrame(prediction, columns=['prob_present','prob_not_present']) 

y_test = pd.DataFrame(y_test,columns=['present','not_present'])

y_pred = y_test >> bind_cols(prediction_df)

y_pred >>= mutate(actual = case_when([y_pred.present == 1, 'present'],
[y_pred.not_present == 1, 'not_present']),
predicted = case_when([y_pred.prob_present > y_pred.prob_not_present, 'present'],
[y_pred.prob_present < y_pred.prob_not_present, 'not_present']))

y_pred >>= mutate(correct = case_when([y_pred.actual == y_pred.predicted, 'yes'],
[y_pred.actual != y_pred.predicted, 'no']))
y_pred



,imf_1_feature_1,imf_1_feature_2,imf_1_feature_3,imf_1_feature_4,imf_1_feature_5,imf_1_feature_6,imf_1_feature_7,imf_1_feature_8,imf_1_feature_9,imf_1_feature_10,...,imf_1_feature_248,imf_1_feature_249,imf_1_feature_250,imf_1_feature_251,imf_1_feature_252,imf_1_feature_253,imf_1_feature_254,imf_1_feature_255,imf_1_feature_256,drone_present
0,0.048540,0.104127,0.036859,0.035103,0.015363,0.086067,0.043169,0.036946,0.072398,0.100767,...,0.054025,0.091116,0.050495,0.020504,0.040957,0.022595,0.082013,0.097442,0.005685,yes
1,0.019087,0.015032,0.016774,0.098352,0.047071,0.070620,0.070484,0.022026,0.093699,0.023186,...,0.100857,0.098248,0.071655,0.070450,0.078421,0.041970,0.039761,0.063975,0.037318,yes
2,0.105886,0.028141,0.012870,0.016689,0.102190,0.129517,0.075366,0.056435,0.094701,0.037966,...,0.071534,0.099206,0.082002,0.077314,0.037580,0.078382,0.105992,0.176923,0.034027,yes
3,0.055652,0.020754,0.084426,0.068263,0.045849,0.075612,0.094904,0.053560,0.086233,0.072574,...,0.041053,0.060956,0.083080,0.046290,0.019449,0.126770,0.075964,0.072927,0.073517,yes
4,0.055740,0.048381,0.059448,0.049503,0.012088,0.076470,0.126046,0.088670,0.066795,0.046619,...,0.053259,0.037194,0.044701,0.059681,0.041496,0.084235,0.060550,0.082656,0.058736,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,0.032114,0.018314,0.009516,0.031732,0.016047,0.008095,0.028272,0.018574,0.017942,0.043779,...,0.041790,0.020034,0.017063,0.027498,0.007705,0.015313,0.033260,0.010302,0.018391,no
3196,0.007821,0.023563,0.013839,0.021590,0.009250,0.016283,0.013973,0.009530,0.015082,0.017309,...,0.016522,0.016841,0.008755,0.013591,0.015497,0.008827,0.022630,0.014762,0.023727,no
3197,0.004704,0.012574,0.031253,0.015620,0.002208,0.029783,0.025996,0.007936,0.008834,0.008706,...,0.008310,0.009864,0.007290,0.025284,0.028346,0.002107,0.016373,0.032724,0.012556,no
3198,0.010071,0.009639,0.030514,0.010773,0.001604,0.018141,0.030366,0.017224,0.009127,0.020327,...,0.019404,0.010192,0.015823,0.029535,0.017265,0.001531,0.011292,0.031961,0.009573,no


In [ ]:
# Hi Riley, if you can read this, then you have done everything right... Good Job!